# Test the split
Notebook for testing SelectSplitData and prepare a new way for splitting the data

In [ ]:
%load_ext autoreload
%autoreload 2

import sys

# setting path
sys.path.append('../')

from modules.TransformApplier import TransformApplier
from modules.preprocessing import Wav2Spec
from modules.data import SimpleDataset
from modules.modelling import *
#from modules.OnlyXTransform import OnlyXTransform
from modules.preprocessing import *
from modules.postprocessing import *

import pandas as pd
import torch.nn as nn
import torch
import timm
import json
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np



In [ ]:
def collate_fn(data):
    """
    Define how the DataLoaders should batch the data
    """
    max_dim = max([d[0].shape[-1] for d in data])
    pad_x = lambda x: torch.concat([x, torch.zeros((max_dim - x.shape[-1], ))])
    return torch.stack([pad_x(d[0]) for d in data], axis=0), torch.stack([torch.tensor(d[1]) for d in data])
    

In [3]:
DATA_PATH = '../data/'
metadata = pd.read_csv(f'{DATA_PATH}train_metadata.csv')[:4000]

with open(f'{DATA_PATH}all_birds.json') as f:
    birds = json.load(f)

In [4]:
tts = metadata.sample(frac=.05).index # train test split
df_val = metadata.iloc[tts]
df_train = metadata.drop(axis=0,index=tts)

In [5]:
df_val.shape

(200, 13)

In [6]:
train_data = SimpleDataset(df_train, DATA_PATH, mode='train', labels=birds)
val_data = SimpleDataset(df_val, DATA_PATH, mode='train', labels=birds)

In [7]:
len(val_data)/16

12.5

In [8]:
bs = 16
train_loader = DataLoader(train_data, batch_size=bs, num_workers=0, collate_fn=collate_fn, shuffle=True)
val_loader = DataLoader(val_data, batch_size=bs, num_workers=0, collate_fn=collate_fn)
# 1 row per batch: 47 rows for val, 882 for train


In [9]:
for x,y in val_loader:
    print(x.shape[1])

6485760
4161792
3730688
3468288
FAIL READING rec ../data/train_audio/cangoo/XC595412.ogg


UnboundLocalError: local variable 'wav' referenced before assignment

In [ ]:
 inputs, classes = next(iter(val_loader))  

In [ ]:
inputs.shape

In [ ]:
x_v, y_v = data_pipeline_val((inputs.to(device),classes.to(device).float()))

In [ ]:
inputs.shape[-1]/16000

In [ ]:
30*16000

In [ ]:
duration = 30
n_splits = 5
sr = 16000
total_duration = inputs.shape[-1] / sr
max_offset = total_duration - duration 
offset = uniform(low=0.0, high=max_offset)
start = int(offset*sr)
stop = min([int((offset + duration)*sr), inputs.shape[-1]])
print(start,stop)
print(stop - start )

In [ ]:
inputs = inputs[..., start:stop]

In [ ]:
inputs.shape

In [ ]:
inputs.reshape((inputs.shape[0]*n_splits, *inputs.shape[1:-1], -1)).shape
# x_v

In [ ]:
x_v.shape

In [ ]:
min(0,2)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
duration = 30
n_splits = 5
transforms1 = TransformApplier([nn.Identity(), SelectSplitData(duration, n_splits)])
data_pipeline_val = nn.Sequential(transforms1).to(device)


In [ ]:
for x_v,y_v in val_loader:
    x_v, y_v = data_pipeline_val((x_v.to(device),y_v.to(device)).float)
